In [1]:
import pandas as pd
from flags import FLAGS, unparsed
import numpy as np
from db.conn_db import db,cursor,engine,truncate_table,data_from_mysql,dev_id_train
import operator  
from functools import reduce
import logging
from sklearn.utils import shuffle


import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from sklearn.cross_validation import KFold
%matplotlib inline


/home/zhijiehuang/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(filename)s:%(lineno)d - %(message)s', level=logging.DEBUG)

In [3]:
deviceid_brand=pd.read_csv(FLAGS.file_path+'deviceid_brand.csv')

In [4]:
deviceid_brand['brand']=deviceid_brand.brand.fillna('unknown')

In [5]:
english_phone_brands_mapping = { "三星": "samsung", "天语": "Ktouch", "海信": "hisense", "联想": "lenovo", "欧比": "obi", "爱派尔": "ipair", "努比亚": "nubia", "优米": "youmi", "朵唯": "dowe", "黑米": "heymi", "锤子": "hammer", "酷比魔方": "koobee", "美图": "meitu", "尼比鲁": "nibilu", "一加": "oneplus", "优购": "yougo", "诺基亚": "nokia", "糖葫芦": "candy", "中国移动": "ccmc", "语信": "yuxin", "基伍": "kiwu", "青橙": "greeno", "华硕": "asus", "夏新": "panosonic", "维图": "weitu", "艾优尼": "aiyouni", "摩托罗拉": "moto", "乡米": "xiangmi", "米奇": "micky", "大可乐": "bigcola", "沃普丰": "wpf", "神舟": "hasse", "摩乐": "mole", "飞秒": "fs", "米歌": "mige", "富可视": "fks", "德赛": "desci", "梦米": "mengmi", "乐视": "lshi", "小杨树": "smallt", "纽曼": "newman", "邦华": "banghua", "E派": "epai", "易派": "epai", "普耐尔": "pner", "欧新": "ouxin", "西米": "ximi", "海尔": "haier", "波导": "bodao", "糯米": "nuomi", "唯米": "weimi", "酷珀": "kupo", "谷歌": "google", "昂达": "ada", "聆韵": "lingyun", "小米": "Xiaomi", "华为": "Huawei", "魅族": "Meizu", "中兴": "ZTE", "酷派": "Coolpad", "金立": "Gionee", "SUGAR": "SUGAR", "OPPO": "OPPO", "vivo": "vivo", "HTC": "HTC", "LG": "LG", "ZUK": "ZUK", "TCL": "TCL", "LOGO": "LOGO", "SUGAR": "SUGAR", "Lovme": "Lovme", "PPTV": "PPTV", "ZOYE": "ZOYE", "MIL": "MIL", "索尼" : "Sony", "欧博信" : "Opssom", "奇酷" : "Qiku", "酷比" : "CUBE", "康佳" : "Konka", "亿通" : "Yitong", "金星数码" : "JXD", "至尊宝" : "Monkey King", "百立丰" : "Hundred Li Feng", "贝尔丰" : "Bifer", "百加" : "Bacardi", "诺亚信" : "Noain", "广信" : "Kingsun", "世纪天元" : "Ctyon", "青葱" : "Cong", "果米" : "Taobao", "斐讯" : "Phicomm", "长虹" : "Changhong", "欧奇" : "Oukimobile", "先锋" : "XFPLAY", "台电" : "Teclast", "大Q" : "Daq", "蓝魔" : "Ramos", "奥克斯" : "AUX" ,"索爱":"sony", \
                               '华为':'huawei','清华同方':'qinghuatongfang','小霸王':'subor'} 
#deviceid_brand.brand.map(pd.Series(english_phone_brands_mapping), na_action='ignore')

In [6]:
deviceid_brand['brand']=deviceid_brand['brand'].apply(lambda x:english_phone_brands_mapping[x] if x in english_phone_brands_mapping else x)
deviceid_brand['brand']=deviceid_brand['brand'].map(str.strip)
deviceid_brand['brand']=deviceid_brand['brand'].map(str.lstrip)
deviceid_brand['brand']=deviceid_brand['brand'].map(str.rstrip)
deviceid_brand['brand']=deviceid_brand['brand'].map(str.lower)

In [7]:
deviceid_brand.brand=deviceid_brand.brand.apply(lambda x:x.replace('+','-'))
deviceid_brand.brand=deviceid_brand.brand.apply(lambda x:x.replace('_','-'))
deviceid_brand.brand=deviceid_brand.brand.apply(lambda x:x.replace('*',''))
deviceid_brand.brand=deviceid_brand.brand.apply(lambda x:x.replace('.',''))

In [8]:
deviceid_brand['ori_brand']=deviceid_brand.brand.values


In [9]:
deviceid_brand.loc[deviceid_brand.ori_brand.values=='and','brand']='unknown'
deviceid_brand.loc[deviceid_brand.ori_brand.values=='i%d0%a0hone-9s','brand']='iphone'
deviceid_brand.loc[deviceid_brand.ori_brand.values=='6s','brand']='iphone'
deviceid_brand.loc[deviceid_brand.ori_brand.values=='6plus','brand']='iphone'
deviceid_brand.loc[deviceid_brand.ori_brand.values=='china%2bunicom','brand']='chinaunicom'
deviceid_brand.loc[deviceid_brand.ori_brand.values=='china','brand']='unknown'
deviceid_brand.loc[deviceid_brand.ori_brand.values=='china_mobile','brand']='chinamobile'  
deviceid_brand.loc[deviceid_brand.ori_brand.values=='移动4g','brand']='chinamobile' 
deviceid_brand.loc[deviceid_brand.ori_brand.values=='OINOM%2BV9%252B','brand']='unknown'
deviceid_brand.loc[deviceid_brand.ori_brand.values=='r2017','brand']='oppo'
deviceid_brand.loc[deviceid_brand.ori_brand.values=='r2010','brand']='oppo'
deviceid_brand.loc[deviceid_brand.ori_brand.values=='thtf(tsinghua','brand']='tsinghua'
deviceid_brand.loc[deviceid_brand.ori_brand.values=='hedy(one','brand']='hedy'
deviceid_brand.loc[deviceid_brand.ori_brand.values=='uimi(z)','brand']='uimi'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('201'),'brand']='hongmi'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('uimi'),'brand']='uimi'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('changhong'),'brand']='changhong'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('xiangmi'),'brand']='xiaomi'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('ximi'),'brand']='xiaomi'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('galaxy'),'brand']='samsung'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('galaxy'),'brand']='samsung'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('motorola'),'brand']='moto'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('yq'),'brand']='yq'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('r80'),'brand']='oppo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('r81'),'brand']='oppo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('r7'),'brand']='oppo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('m3'),'brand']='meizu'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('v1'),'brand']='gionee'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('gn'),'brand']='gionee'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('l50u'),'brand']='sony'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('hs'),'brand']='hisense'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('h3'),'brand']='huawei'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('h1'),'brand']='huawei'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('h6'),'brand']='huawei'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('n1'),'brand']='oppo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('sc-'),'brand']='htc'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('f3'),'brand']='gionee'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('scl23'),'brand']='samsung'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('a31t'),'brand']='oppo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('sm'),'brand']='smartisan'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('mx4'),'brand']='meizu'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('q11'),'brand']='htc'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('3007'),'brand']='oppo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('c730lw'),'brand']='phicomm'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('c6'),'brand']='sony'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('c6'),'brand']='sony'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('l3'),'brand']='sony'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('x8'),'brand']='sony'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('n918st'),'brand']='zte'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('x1'),'brand']='vivo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('k-touch'),'brand']='ktouch'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('8848'),'brand']='ktouch'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('tetc'),'brand']='nokia'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('g621'),'brand']='huawei'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('x-apple'),'brand']='iphone'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('r8'),'brand']='oppo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('q-note'),'brand']='qnote'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('q-note'),'brand']='qnote'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('s8'),'brand']='samsung'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('nx507j'),'brand']='nubia'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('nx403a'),'brand']='nubia'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('nx505h'),'brand']='nubia'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('k88l'),'brand']='pioneer'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('k8'),'brand']='lg'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('yd206'),'brand']='yd'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('v300'),'brand']='gionee'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('v9180'),'brand']='zte'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('v615'),'brand']='konka'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('v919'),'brand']='omda'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('mediapad'),'brand']='huawei'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('u707t'),'brand']='oppo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('z1988'),'brand']='oledee'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('g620'),'brand']='huawei'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('w9'),'brand']='gionee'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('1100'),'brand']='nubia'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('daq'),'brand']='zte'
deviceid_brand.loc[deviceid_brand.ori_brand.values=='东山科技','brand']='unknown'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('3000'),'brand']='lg'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('1105'),'brand']='oppo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('8295'),'brand']='coolpad'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('cool'),'brand']='coolpad'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('a11'),'brand']='htc'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('a1'),'brand']='oppo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('a1001'),'brand']='oneplus'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('a0001'),'brand']='oneplus'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('sk3'),'brand']='oneplus'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('ideataba1010'),'brand']='lenovo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('m04'),'brand']='meizu'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('x9'),'brand']='oppo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('xm'),'brand']='sony'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('s55u'),'brand']='sony'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('ds530m'),'brand']='honmii'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('l55u'),'brand']='honmii'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('l55u'),'brand']='honmii'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('s39h'),'brand']='honmii'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('la'),'brand']='xiaolajiao'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('w20'),'brand']='sony'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('i60'),'brand']='lenovo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('j5'),'brand']='samsung'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('a7'),'brand']='samsung'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('r12'),'brand']='oppo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('n5'),'brand']='oppo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('c856'),'brand']='cappu'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('a6'),'brand']='konka'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('meitu'),'brand']='meitu'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('n958st'),'brand']='zte'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('a31c'),'brand']='oppo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('a368'),'brand']='lenovo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('u3'),'brand']='oppo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('e6'),'brand']='gionee'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('gt001'),'brand']='unknown'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('q8000'),'brand']='changhong'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('k706'),'brand']='unknown'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('e7'),'brand']='samsung'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('sk'),'brand']='coolpad'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('htl23'),'brand']='htc'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('k1001'),'brand']='lenovo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('100'),'brand']='100jia'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('mzt'),'brand']='mzt'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('semc'),'brand']='sony'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('cmcc'),'brand']='chinamobile'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('mc002'),'brand']='21ke'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('k0'),'brand']='asus'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('k3'),'brand']='lg'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('e3'),'brand']='meizu'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('d2533'),'brand']='sony'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('d2533'),'brand']='sony'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('mtk6589'),'brand']='mtk'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('u9'),'brand']='moto'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('g9'),'brand']='huawei'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('g8802'),'brand']='ozzo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('t8620'),'brand']='huawei'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('n800i'),'brand']='nubia'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('mf66'),'brand']='unknown'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('f5d'),'brand']='unknown'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('m19'),'brand']='unknown'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('lg'),'brand']='lg'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('t9'),'brand']='oppo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('t200'),'brand']='coolpad'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('t610a'),'brand']='coolpad'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('r5'),'brand']='oppo'

In [10]:
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('w800'),'brand']='konka'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('mt788'),'brand']='moto'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('d6653'),'brand']='sony'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('m51w'),'brand']='sony'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('sl600'),'brand']='samsung'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('m812c'),'brand']='chinamobile'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('m812c'),'brand']='tcl'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('g0111'),'brand']='geli'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('k18'),'brand']='gionee'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('f5180'),'brand']='hisense'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('8085n'),'brand']='coolpad'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('1phone'),'brand']='iphone'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('lte4g'),'brand']='chinamobile'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('a906'),'brand']='tcl'

deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('r5'),'brand']='oppo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('cm9'),'brand']='unknown'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('r6007'),'brand']='oppo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('n900'),'brand']='coolpad'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('dkl01'),'brand']='dakele'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('i9300'),'brand']='samsung'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('i999'),'brand']='philips'

In [11]:
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('1080p'),'brand']='unknown'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('a5'),'brand']='lenovo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('m623c'),'brand']='chinamobile'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('亿美q007'),'brand']='eoom'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('z530'),'brand']='acer'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('4g'),'brand']='chinamobile'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('che'),'brand']='huawei'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('a31'),'brand']='oppo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('6607'),'brand']='oppo'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('t1'),'brand']='smartisan'
deviceid_brand.loc[deviceid_brand['ori_brand'].str.contains('1107'),'brand']='samsung'

In [12]:
deviceid_brand['brand']=deviceid_brand.brand.apply(lambda x:x.split('-')[0])
deviceid_brand['brand']=deviceid_brand.brand.apply(lambda x:x.split('%')[0])

deviceid_brand[deviceid_brand['brand'].str.contains('la')].brand.unique()

t= deviceid_brand[deviceid_brand['ori_brand'].str.contains('xxxxx')]
t

brand=deviceid_brand.brand.value_counts()[deviceid_brand.brand.value_counts()>10].index.tolist()
i=0
for x in  brand:
    if  x.isalnum() and not x.isalpha():
        print(i,x)
        i=i+1

deviceid_train=pd.read_csv(FLAGS.file_path+'deviceid_train.csv')
deviceid_train=pd.merge(deviceid_train,deviceid_brand,on=['device_id'],how='left')

deviceid_test=pd.read_csv(FLAGS.file_path+'deviceid_test.csv')
deviceid_test=pd.merge(deviceid_test,deviceid_brand,on=['device_id'],how='left')

deviceid_test.brand.value_counts()[deviceid_test.brand.value_counts()<10].index.tolist()

deviceid_packages=pd.read_csv(FLAGS.file_path+'deviceid_packages.csv')

deviceid_train=dev_id_train()

deviceid_train=pd.merge(deviceid_packages,deviceid_train,on=['device_id'],how='left') 
deviceid_train=pd.merge(deviceid_train,deviceid_brand,on=['device_id'],how='left') 

def data_ix(deviceid_train,class_n):
    
    filte=deviceid_train.n_class.values==class_n
    tmp_deviceid_train=deviceid_train.ix[filte,:]
    return tmp_deviceid_train

deviceid_train_1=data_ix(deviceid_train,1)

deviceid_train_1.brand.unique()



fig,ax = plt.subplots(figsize=(8,6))

deviceid_train_1.brand.value_counts().plot(kind='barh')
plt.title('Age Group Distribution')
plt.xlabel('Count')



# 2 type_no

deviceid_brand=pd.read_csv(FLAGS.file_path+'deviceid_brand.csv')

In [13]:
deviceid_brand['type_no']=deviceid_brand.type_no.fillna('unknown')
deviceid_brand['type_no']=deviceid_brand['type_no'].map(str.strip)
deviceid_brand['type_no']=deviceid_brand['type_no'].map(str.lstrip)
deviceid_brand['type_no']=deviceid_brand['type_no'].map(str.rstrip)
deviceid_brand['type_no']=deviceid_brand['type_no'].map(str.lower)

In [14]:
deviceid_brand['ori_type_no']=deviceid_brand.type_no.values

In [15]:
deviceid_brand.type_no=deviceid_brand.type_no.apply(lambda x:x.replace('coolpad',''))
deviceid_brand.type_no=deviceid_brand.type_no.apply(lambda x:x.replace('changhong',''))
deviceid_brand.type_no=deviceid_brand.type_no.apply(lambda x:x.replace('samsung',''))
deviceid_brand.type_no=deviceid_brand.type_no.apply(lambda x:x.replace('htc',''))

In [16]:
deviceid_brand.type_no=deviceid_brand.type_no.apply(lambda x:x.replace('+','_'))
deviceid_brand.type_no=deviceid_brand.type_no.apply(lambda x:x.replace('_','_'))
deviceid_brand.type_no=deviceid_brand.type_no.apply(lambda x:x.replace('/','_'))
deviceid_brand.type_no=deviceid_brand.type_no.apply(lambda x:x.replace('*',''))
deviceid_brand.type_no=deviceid_brand.type_no.apply(lambda x:x.replace('.','_'))

In [17]:
def type_no_cut(brand,type_no):
    type_no_list=type_no.split('_')
    if len(type_no_list)<2:
        return type_no_list[0]
    for x in  type_no_list:
#        if x.isdigit():
#            print(brand,type_no)
        x1=x.replace('-','')
        if  x1.isalnum() and not x1.isalpha():
            return x
    return brand

In [18]:
deviceid_brand.type_no=deviceid_brand.apply(lambda line:type_no_cut(line['brand'],line['type_no']),axis=1)

In [19]:
deviceid_brand.type_no.value_counts().index.tolist()

['hm',
 'vivo',
 'huawei',
 '',
 'mi',
 'lenovo',
 'oppo',
 'zte',
 'h60-l02',
 'gt-n7100',
 'che2-ul00',
 'pe-tl10',
 '2014811',
 'che1-cl20',
 '2014501',
 'chm-ul00',
 'sm-n9006',
 'pe-ul00',
 'sm-n9002',
 'gt-n7102',
 'sm-a7000',
 'r8207',
 'sm-a5000',
 '2014813',
 'a31',
 'h30-u10',
 'sm-g7106',
 'gt-i9500',
 '2014011',
 'h60-l12',
 'sm-n900',
 'm1',
 'chm-tl00h',
 'tcl',
 'chm-tl00',
 '2013022',
 'sm-g7508q',
 'sm-g7200',
 'r8200',
 'note',
 'la2-sn',
 'g620s-ul00',
 'la2-s',
 'sm-n7506v',
 'gt-i9152',
 'gt-i9300i',
 '2013023',
 'gt-i9300',
 'sm-n9005',
 'a31u',
 'n5117',
 'h60-l01',
 'che2-tl00m',
 '1107',
 'gt-i8552',
 'yq601',
 'm351',
 'doov',
 'l50u',
 'x9007',
 'sm-g9006v',
 'gt-n7108',
 'sm-e7000',
 'che2-tl00',
 'r8107',
 'lephone',
 'gt-n5100',
 'honor',
 'v183',
 'a11',
 'gt-i9082i',
 'hol-t00',
 'sm-n9100',
 'c730lw',
 'sm-g5308w',
 '6607',
 'sm705',
 'r2017',
 'n5207',
 'g621-tl00',
 'a31t',
 'gt-i9502',
 'gt-i9152p',
 'sm-g5108q',
 'pe-tl00m',
 'pe-tl20',
 'h60-l03',


In [22]:
deviceid_brand.head(5)

,brand,device_id,type_no,ori_brand,ori_type_no
0,samsung,f69cdc2f5018aa6d3d6500976b926c09,gt-n7100,samsung,gt-n7100
1,samsung,522690089a93278b65025127d937e9bb,gt-i9507v,samsung,gt-i9507v
2,sony,dcf13c9ec255153bfa909dc3c64908c3,l39u,sony,l39u
3,sony,58b5f0ec136407caf3708a4b05c1894f,l39u,sony,l39u
4,sony,dbca7cb790645b18383f4536e7f1f303,xm50h,sony,xm50h


In [23]:
deviceid_brand.to_csv(FLAGS.file_path+'new_deviceid_brand.csv',index= False)